In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import acr
import xarray as xr
from scipy import stats
# ---------------------------- EXTRAS --------------------------------#
from kdephys.plot.main import _title, bp_plot
import kdephys.utils.spectral as sp
bands = sp.bands
from scipy.stats import normaltest
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
from acr.utils import *
pu = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pu')
dag = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'dag')
import pingouin as pg
from scipy.stats import shapiro

plt.rcdefaults()
acr.plots.lrg()

In [ ]:
%connect_info

In [ ]:
from acr.utils import SOM_BLUE, ACR_BLUE, NNXR_GRAY

MAIN_EXP = 'swi'
SUBJECT_TYPE = 'control'
MAIN_COLOR = CTRL_BLUE

In [ ]:
notebook_figure_root = f'{pu.PAPER_FIGURE_ROOT}/synchrony/field_potentials'
subjects, exps = pu.get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)

In [ ]:
full_hyps = {}
hyp_dicts = {}
for subject, exp in zip(subjects, exps):
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, exp)
    hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=True, duration='1800s')

In [ ]:
# Load the Full Exp field potentials
raw_lfps = {}
for subject, exp in zip(subjects, exps):
        if subject not in raw_lfps.keys():
            recordings = acr.info_pipeline.get_exp_recs(subject, exp)
            for rec in recordings:
                if '-post' in rec:
                    recordings.remove(rec)
            print(subject, recordings)
            lfp = acr.io.load_concat_raw_data(subject, recordings)
            raw_lfps[subject] = lfp

# Load OFFs

In [ ]:
bindfs = []
oodfs = {}
for subject, exp in zip(subjects, exps):
        print(subject, exp)
        if subject in bindfs:
                oodf = dag.load_bindf_with_hybrid_format(subject, exp)
        else:
                oodf = dag.compute_hybrid_off_df(subject, exp, chan_threshold=12)
        oodf = acr.oo_utils.enhance_oodf(oodf, full_hyps[subject], hyp_dicts[subject])
        oodfs[subject] = oodf
full_oodf = pl.concat(oodfs.values())
off_means = full_oodf.group_by(['subject', 'probe', 'condition']).agg(pl.col('duration_rel').mean())

In [ ]:
bindfs = []
oodfs = {}
for subject, exp in zip(subjects, exps):
        oodf = dag.load_base_oodf(subject, exp)
        oodf = acr.oo_utils.enhance_oodf(oodf, full_hyps[subject], hyp_dicts[subject])
        oodfs[subject] = oodf
full_oodf = pl.concat(oodfs.values())
off_means = full_oodf.group_by(['subject', 'probe', 'condition']).agg(pl.col('duration_rel').mean())

In [ ]:
from acr.sync import *
# ____________________________SIMPLE CONDITION AVERAGES _____________________________
#=====================================================================================
raw_term_data = {}
# -----------------------------------------------------------------------------------
# ---------------------- OFF-Termination Slope ----------------------------------------
# -----------------------------------------------------------------------------------
cdn_dfs = [] # slope dataframes for each subject, condition, probe
for subject in raw_lfps.keys():
    raw_term_data[subject] = {}
    for probe in ['NNXo', 'NNXr']:
        print(subject, probe)
        raw_term_data[subject][probe] = {}
        data = raw_lfps[subject].prb(probe)
        oodf = oodfs[subject].prb(probe)
        for condition in COND_ORDER:
            off_ends = oodf.cdn(condition)['end_datetime'].to_pandas()
            if len(off_ends) == 0:
                continue
            term_data = select_data_around_times(off_ends, data, buffer=0.10)
            raw_term_data[subject][probe][condition] = term_data
            term_slopes = compute_slope_df(term_data, data.fs, find='min', window_length=15, polyorder=5, search_buffer=10)
            term_slopes['condition'] = condition
            term_slopes['probe'] = probe
            term_slopes['subject'] = subject
            cdn_dfs.append(term_slopes)
        for full_bl_val in ['True']:
            off_ends = oodf.filter(pl.col('full_bl')==full_bl_val)['end_datetime'].to_pandas()
            if len(off_ends) == 0:
                continue
            term_data = select_data_around_times(off_ends, data, buffer=0.10)
            raw_term_data[subject][probe]['full_bl'] = term_data
            term_slopes = compute_slope_df(term_data, data.fs, find='min', window_length=15, polyorder=5, search_buffer=10)
            term_slopes['condition'] = 'full_bl'
            term_slopes['probe'] = probe
            term_slopes['subject'] = subject
            cdn_dfs.append(term_slopes)
all_term_slopes = pd.concat(cdn_dfs)
all_term_slopes['chan_str'] = all_term_slopes['channel'].astype(str)

# -----------------------------------------------------------------------------------
# ---------------------- OFF-Initiation Slope ----------------------------------------
# -----------------------------------------------------------------------------------
raw_init_data = {}
cdn_dfs = [] # slope dataframes for each subject, condition, probe
for subject in raw_lfps.keys():
    raw_init_data[subject] = {}
    for probe in ['NNXo', 'NNXr']:
        raw_init_data[subject][probe] = {}
        data = raw_lfps[subject].prb(probe)
        oodf = oodfs[subject].prb(probe)
        for condition in COND_ORDER:
            off_starts = oodf.cdn(condition)['start_datetime'].to_pandas()
            if len(off_starts) == 0:
                continue
            init_data = select_data_around_times(off_starts, data, buffer=0.10)
            raw_init_data[subject][probe][condition] = init_data
            init_slopes = compute_slope_df(init_data, data.fs, find='max', window_length=15, polyorder=5, search_buffer=10)
            init_slopes['condition'] = condition
            init_slopes['probe'] = probe
            init_slopes['subject'] = subject
            cdn_dfs.append(init_slopes)
        for full_bl_val in ['True']:
            off_starts = oodf.filter(pl.col('full_bl')==full_bl_val)['start_datetime'].to_pandas()
            if len(off_starts) == 0:
                continue
            init_data = select_data_around_times(off_starts, data, buffer=0.10)
            raw_init_data[subject][probe]['full_bl'] = init_data
            init_slopes = compute_slope_df(init_data, data.fs, find='max', window_length=15, polyorder=5, search_buffer=10)
            init_slopes['condition'] = 'full_bl'
            init_slopes['probe'] = probe
            init_slopes['subject'] = subject
            cdn_dfs.append(init_slopes)
all_init_slopes = pd.concat(cdn_dfs)
all_init_slopes['chan_str'] = all_init_slopes['channel'].astype(str)

In [ ]:
# ____________________________CONDITION x DURATION AVERAGES _____________________________
#=====================================================================================
dur_buckets = np.arange(0.05, 0.22, 0.02)

# -----------------------------------------------------------------------------------
# ---------------------- OFF-Termination Slope ----------------------------------------
# -----------------------------------------------------------------------------------
cdn_dfs = [] # slope dataframes for each subject, condition, probe
for subject in raw_lfps.keys():
    for probe in ['NNXo', 'NNXr']:
        data = raw_lfps[subject].prb(probe)
        oodf = oodfs[subject].prb(probe)
        for condition in COND_ORDER:
            for dur_edge in dur_buckets:
                off_starts = oodf.cdn(condition).filter((pl.col('duration')>=dur_edge)&(pl.col('duration')<dur_edge+0.02))['end_datetime'].to_pandas()
                if len(off_starts) == 0:
                    continue
                term_data = select_data_around_times(off_starts, data, buffer=0.10)
                term_slopes = compute_slope_df(term_data, data.fs, find='min', window_length=15, polyorder=5, search_buffer=10)
                term_slopes['condition'] = condition
                term_slopes['probe'] = probe
                term_slopes['subject'] = subject
                term_slopes['duration'] = dur_edge
                term_slopes['n_offs'] = len(off_starts)
                cdn_dfs.append(term_slopes)
        for full_bl_val in ['True']:
            for dur_edge in dur_buckets:
                off_starts = oodf.filter(pl.col('full_bl')==full_bl_val).filter((pl.col('duration')>=dur_edge)&(pl.col('duration')<dur_edge+0.02))['end_datetime'].to_pandas()
                if len(off_starts) == 0:
                    continue
                term_data = select_data_around_times(off_starts, data, buffer=0.10)
                term_slopes = compute_slope_df(term_data, data.fs, find='min', window_length=15, polyorder=5, search_buffer=10)
                term_slopes['condition'] = 'full_bl'
                term_slopes['probe'] = probe
                term_slopes['subject'] = subject
                term_slopes['duration'] = dur_edge
                term_slopes['n_offs'] = len(off_starts)
                cdn_dfs.append(term_slopes)
term_dur_slopes = pd.concat(cdn_dfs)
term_dur_slopes['chan_str'] = term_dur_slopes['channel'].astype(str)


# -----------------------------------------------------------------------------------
# ---------------------- OFF-Initiation Slope ----------------------------------------
# -----------------------------------------------------------------------------------
cdn_dfs = [] # slope dataframes for each subject, condition, probe
for subject in raw_lfps.keys():
    for probe in ['NNXo', 'NNXr']:
        data = raw_lfps[subject].prb(probe)
        oodf = oodfs[subject].prb(probe)
        for condition in COND_ORDER:
            for dur_edge in dur_buckets:
                off_starts = oodf.cdn(condition).filter((pl.col('duration')>=dur_edge)&(pl.col('duration')<dur_edge+0.02))['start_datetime'].to_pandas()
                if len(off_starts) == 0:
                    continue
                init_data = select_data_around_times(off_starts, data, buffer=0.10)
                init_slopes = compute_slope_df(init_data, data.fs, find='max', window_length=15, polyorder=5, search_buffer=10)
                init_slopes['condition'] = condition
                init_slopes['probe'] = probe
                init_slopes['subject'] = subject
                init_slopes['duration'] = dur_edge
                init_slopes['n_offs'] = len(off_starts)
                cdn_dfs.append(init_slopes)
        for full_bl_val in ['True']:
            for dur_edge in dur_buckets:
                off_starts = oodf.filter(pl.col('full_bl')==full_bl_val).filter((pl.col('duration')>=dur_edge)&(pl.col('duration')<dur_edge+0.02))['start_datetime'].to_pandas()
                if len(off_starts) == 0:
                    continue
                init_data = select_data_around_times(off_starts, data, buffer=0.10)
                init_slopes = compute_slope_df(init_data, data.fs, find='max', window_length=15, polyorder=5, search_buffer=10)
                init_slopes['condition'] = 'full_bl'
                init_slopes['probe'] = probe
                init_slopes['subject'] = subject
                init_slopes['duration'] = dur_edge
                init_slopes['n_offs'] = len(off_starts)
                cdn_dfs.append(init_slopes)
init_dur_slopes = pd.concat(cdn_dfs)
init_dur_slopes['chan_str'] = init_dur_slopes['channel'].astype(str)

In [ ]:
term_slopes = relativize_slope_df_to_condition(all_term_slopes, 'full_bl', on=['subject', 'probe', 'channel'])
init_slopes = relativize_slope_df_to_condition(all_init_slopes, 'full_bl', on=['subject', 'probe', 'channel'])

In [ ]:
dur_term_slopes = relativize_slope_df_to_condition(term_dur_slopes, 'full_bl', on=['subject', 'probe', 'channel', 'duration'])
dur_init_slopes = relativize_slope_df_to_condition(init_dur_slopes, 'full_bl', on=['subject', 'probe', 'channel', 'duration'])

# Initiation and Termination Slopes

In [ ]:
# obvious outlier/artifactual channels for exclusions
scd = {}
#scd['ACR_29'] = [16]
#scd['ACR_37'] = [14, 15, 16]
#scd['ACR_41'] = [1, 2]
# =============================

In [ ]:
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False

In [ ]:
reb_init = init_slopes.cdn('rebound')

for subject in reb_init['subject'].unique():
    reb_init = pu.add_layer_info_to_df(reb_init, subject)

reb_init = pu.drop_sub_channels(reb_init, scd)

init_probe_means = reb_init.groupby(['subject', 'probe']).slope_rel.mean().to_frame()
init_probe_means.reset_index(inplace=True)
init_chan_means = reb_init.groupby(['subject', 'probe', 'channel']).slope_rel.mean().to_frame()
init_chan_means.reset_index(inplace=True)
init_layer_means = reb_init.groupby(['subject', 'probe', 'layer']).slope_rel.mean().to_frame()
init_layer_means.reset_index(inplace=True)

In [ ]:
reb_term = term_slopes.cdn('rebound')
for subject in reb_term['subject'].unique():
    reb_term = pu.add_layer_info_to_df(reb_term, subject)
reb_term = pu.drop_sub_channels(reb_term, scd)
term_probe_means = reb_term.groupby(['subject', 'probe']).slope_rel.mean().to_frame()
term_probe_means.reset_index(inplace=True)
term_chan_means = reb_term.groupby(['subject', 'probe', 'channel']).slope_rel.mean().to_frame()
term_chan_means.reset_index(inplace=True)
term_layer_means = reb_term.groupby(['subject', 'probe', 'layer']).slope_rel.mean().to_frame()
term_layer_means.reset_index(inplace=True)

In [ ]:
#channel-by-channel checks if needed
g = sns.catplot(data=reb_init, x='channel', y='slope_rel', kind='bar', hue='probe', col='subject', col_wrap=2, height=4, aspect=3)
g = sns.catplot(data=reb_term, x='channel', y='slope_rel', kind='bar', hue='probe', col='subject', col_wrap=2, height=4, aspect=3)

In [ ]:
plt.rcdefaults()
acr.plots.supl()

In [ ]:
nnxr = init_probe_means.prb('NNXr')['slope_rel'].values
nnxo = init_probe_means.prb('NNXo')['slope_rel'].values

fig_id = 'rebound-init-slope-all-chans'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, ax=ax, colors=[NNXR_GRAY, MAIN_COLOR])
print(ax.get_ylim())

#ax.set_yticks([0.8, 1, 1.2])
#ax.set_ylim(0.8, 1.26)
ax.set_xticklabels(['Contra Control', 'Optrode'])
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
#stats = pg.wilcoxon(nnxr, nnxo)
stats = pg.ttest(nnxr, nnxo, paired=True)
hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')
#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))
if write:
    stats_name = fig_name
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', [hg])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
rnorm = nnxr/nnxr
onorm = nnxo/nnxr

fig_id = 'rebound-init-slope-all-chans-NORMALIZED'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
f, ax = acr.plots.gen_paired_boxplot(rnorm, onorm, ax=ax, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True)
print(ax.get_ylim())

f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)


In [ ]:
nnxr = term_probe_means.prb('NNXr')['slope_rel'].values
nnxo = term_probe_means.prb('NNXo')['slope_rel'].values

fig_id = 'rebound-term-slope-all-chans'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, ax=ax, colors=[NNXR_GRAY, MAIN_COLOR])
print(ax.get_ylim())

ax.set_xticklabels(['Contra Control', 'Optrode'])
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
stats = pg.ttest(nnxr, nnxo, paired=True)
hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')
#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))
if write:
    stats_name = fig_name
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', [hg])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
rnorm = nnxr/nnxr
onorm = nnxo/nnxr

fig_id = 'rebound-term-slope-all-chans-NORMALIZED'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
f, ax = acr.plots.gen_paired_boxplot(rnorm, onorm, ax=ax, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True)
print(ax.get_ylim())


ax.set_xticklabels(['Contra Control', 'Optrode'])
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)
ax.get_ylim()

# Peaks and Troughs

In [ ]:
scd = {}
scd['ACR_42'] = [4]

In [ ]:
# ____________________________SIMPLE CONDITION AVERAGES _____________________________
#=====================================================================================

# -----------------------------------------------------------------------------------
# ---------------------- OFF-Termination Slope ----------------------------------------
# -----------------------------------------------------------------------------------
peak_dfs = [] # peak dataframes for each subject, condition, probe
trough_dfs = []
raw_peak_data = {}
raw_trough_data = {}
for subject in raw_lfps.keys():
    raw_peak_data[subject] = {}
    raw_trough_data[subject] = {}
    for probe in ['NNXo', 'NNXr']:
        raw_peak_data[subject][probe] = {}
        raw_trough_data[subject][probe] = {}
        data = raw_lfps[subject].prb(probe)
        oodf = oodfs[subject].prb(probe)
        for condition in COND_ORDER:
            off_ends = oodf.cdn(condition)['end_datetime'].to_pandas()
            off_durs = oodf.cdn(condition)['duration'].to_pandas()
            off_starts = oodf.cdn(condition)['start_datetime'].to_pandas()
            if len(off_ends) == 0:
                continue
            term_data = select_data_around_times(off_ends, data, buffer=0.06)
            raw_trough_data[subject][probe][condition] = term_data
            trough_df = compute_trough_df(term_data, data.fs)
            trough_df['condition'] = condition
            trough_df['probe'] = probe
            trough_df['subject'] = subject
            trough_dfs.append(trough_df)

            peak_data = select_data_for_peaks(off_starts, off_durs, data, buffer=0.06)
            raw_peak_data[subject][probe][condition] = peak_data
            peak_df = compute_peak_df(peak_data)
            peak_df['condition'] = condition
            peak_df['probe'] = probe
            peak_df['subject'] = subject
            peak_dfs.append(peak_df)
        
        
        for full_bl_val in ['True']:
            off_ends = oodf.filter(pl.col('full_bl')==full_bl_val)['end_datetime'].to_pandas()
            off_durs = oodf.filter(pl.col('full_bl')==full_bl_val)['duration'].to_pandas()
            off_starts = oodf.filter(pl.col('full_bl')==full_bl_val)['start_datetime'].to_pandas()
            if len(off_ends) == 0:
                continue
            term_data = select_data_around_times(off_ends, data, buffer=0.06)
            raw_trough_data[subject][probe]['full_bl'] = term_data
            trough_df = compute_trough_df(term_data, data.fs)
            trough_df['condition'] = 'full_bl'
            trough_df['probe'] = probe
            trough_df['subject'] = subject
            trough_dfs.append(trough_df)

            peak_data = select_data_for_peaks(off_starts, off_durs, data, buffer=0.06)
            raw_peak_data[subject][probe]['full_bl'] = peak_data
            peak_df = compute_peak_df(peak_data)
            peak_df['condition'] = 'full_bl'
            peak_df['probe'] = probe
            peak_df['subject'] = subject
            peak_dfs.append(peak_df)

trough_df = pd.concat(trough_dfs)
trough_df['chan_str'] = trough_df['channel'].astype(str)

peak_df = pd.concat(peak_dfs)
peak_df['chan_str'] = peak_df['channel'].astype(str)

In [ ]:
tdf = relativize_slope_df_to_condition(trough_df, 'full_bl', on=['subject', 'probe', 'channel'], col_to_rel='trough')
pdf = relativize_slope_df_to_condition(peak_df, 'full_bl', on=['subject', 'probe', 'channel'], col_to_rel='peak')

In [ ]:
reb_peaks = pdf.cdn('rebound')
reb_peaks = pu.drop_sub_channels(reb_peaks, scd)
for subject in reb_peaks['subject'].unique():
    reb_peaks = pu.add_layer_info_to_df(reb_peaks, subject)

reb_peaks_probe = reb_peaks.groupby(['subject', 'probe']).peak_rel.mean().to_frame().reset_index()
reb_peaks_channel = reb_peaks.groupby(['subject', 'probe', 'channel']).peak_rel.mean().to_frame().reset_index()
reb_peaks_layer = reb_peaks.groupby(['subject', 'probe', 'layer']).peak_rel.mean().to_frame().reset_index()

In [ ]:
reb_troughs = tdf.cdn('rebound')
reb_troughs = pu.drop_sub_channels(reb_troughs, scd)
for subject in reb_troughs['subject'].unique():
    reb_troughs = pu.add_layer_info_to_df(reb_troughs, subject)
reb_troughs_probe = reb_troughs.groupby(['subject', 'probe']).trough_rel.mean().to_frame().reset_index()
reb_troughs_channel = reb_troughs.groupby(['subject', 'probe', 'channel']).trough_rel.mean().to_frame().reset_index()
reb_troughs_layer = reb_troughs.groupby(['subject', 'probe', 'layer']).trough_rel.mean().to_frame().reset_index()

In [ ]:
g = sns.catplot(data=reb_peaks, x='channel', y='peak_rel', hue='probe', kind='bar', col='subject', col_wrap=2, height=4, aspect=3)
g = sns.catplot(data=reb_troughs, x='channel', y='trough_rel', hue='probe', kind='bar', col='subject', col_wrap=2, height=4, aspect=3)

In [ ]:
nnxr = reb_peaks_probe.prb('NNXr')['peak_rel'].values
nnxo = reb_peaks_probe.prb('NNXo')['peak_rel'].values

fig_id = 'rebound-peak_amplitude-all-chans'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, ax=ax, colors=[NNXR_GRAY, MAIN_COLOR])
print(ax.get_ylim())
ax.set_xticklabels(['Contra Control', 'Optrode'])
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
stats = pg.ttest(nnxr, nnxo, paired=True)
hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

if write:
    stats_name = fig_name
    acr.stats.write_stats_result(stats_name, 'paired-ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
rnorm = nnxr/nnxr
onorm = nnxo/nnxr

fig_id = 'rebound-peak-amplitude-all-chans-NORMALIZED'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(3.5, 4))
f, ax = acr.plots.gen_paired_boxplot(rnorm, onorm, ax=ax, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True)
print(ax.get_ylim())
ax.set_xticklabels(['Contra Control', 'Optrode'])
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# Plot direct Potentials

In [ ]:
from scipy.stats import sem

In [ ]:
sub = 'ACR_48'
probe = 'NNXr'
cond = 'full_bl'
rd = raw_peak_data[sub][probe][cond]
mrd = rd.mean(axis=2)
idf = compute_peak_df(rd)
sems = sem(rd, axis=2)
half = mrd.shape[1]//2

In [ ]:
mrd.shape

In [ ]:
f, axs = kde.plot.main.atomic_lfp(mrd, times=None, figsize=(6, 18), hspace=-0.7, sems=sems)
for chan in np.arange(1, 17):
    axn = axs[chan-1]
    axn.axvline(x=half, color='k', linestyle='--')
    position = idf.loc[idf['channel']==chan]['peak_position'].values[0]
    rel_position = half+position
    axn.axvline(x=rel_position, color='r', linestyle='--')

In [ ]:
compute_slope_df()

In [ ]:
rd.shape

In [ ]:
sub = 'ACR_30'
probe = 'NNXo'
cond = 'rebound'
fs = raw_lfps[sub].fs
rd = raw_init_data[sub][probe][cond]
sems = sem(rd, axis=2)
mrd = rd.mean(axis=2)
idf = compute_slope_df(rd, fs, find='max', search_buffer=10)
half = rd.shape[1]/2

In [ ]:
f, axs = kde.plot.main.atomic_lfp(mrd, times=None, figsize=(6, 18), hspace=-0.7, sems=sems)
for chan in np.arange(1, 17):
    axn = axs[chan-1]
    axn.axvline(x=half, color='k', linestyle='--')
    position = idf.loc[idf['channel']==chan]['extrema_rel_position'].values[0]
    rel_position = half+position
    axn.axvline(x=rel_position, color='r', linestyle='--')
    axn.set_xlim(half-15, half+15)